
Candinate Number: 42165

The python code could be found in src folder.

In [62]:
from neo4j import GraphDatabase

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j+ssc://a0e1a868.databases.neo4j.io"
AUTH = ('neo4j', 'd-TXsU0-hj_wYRoL2suasNSQdOOoCx86vElCmibvQGg')

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()



Q2A)  This is in the file q2a_property_Graph.png in src. I made few adjustments becauase I exported my data from the DB file in assignment 1. There I had some views which calculated subtotal and grand total based on some attributes. I made subtotal its own node.


Q2B) This is in the file bloom-visualisation.png. There are some nodes especially on the outer radius which does not have relationships. I also had to adjust some of the nodes and edges to make it appropriate according to the data I have my excel files from assignment 1.


In [64]:
from yfiles_jupyter_graphs import GraphWidget
with driver.session(database="neo4j") as session:
    query = """
    MATCH triple=(n)-[r]->(m)
    RETURN triple
    """
    graph = session.run(query).graph()
    session.close()
graph_widget = GraphWidget(graph=graph)
display(graph_widget)

/var/folders/vk/cw2q21sj3l37fm6ls3k4g97c0000gq/T/ipykernel_55865/1613786816.py:2: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session(database="neo4j") as session:


GraphWidget(layout=Layout(height='800px', width='100%'))

Q2C)

In [45]:
def fetch_customers_orders(uri, auth):
    """Fetch customers, their orders, and total paid."""
    query = """
    MATCH (c:Customers)-[:places]->(o:Orders)-[:order_item]->(p:Products)
    RETURN
      c.first_name AS first_name,
      c.last_name AS last_name,
      c.email AS email,
      o.order_id AS order_id,
      o.date AS order_date,
      collect(p.Name) AS product_list,
      o.grand_total AS grand_total
    ORDER BY last_name, order_date
    """
    with driver.session() as session:
        result = session.run(query)
        data = []
        for record in result:
            # Combine first_name and last_name into full_name
            full_name = f"{record['first_name']} {record['last_name']}"
            data.append({
                "full_name": full_name,
                "email": record["email"],
                "order_id": record["order_id"],
                "order_date": record["order_date"],
                "product_list": record["product_list"],
                "grand_total": record["grand_total"],
            })

    return data


if __name__ == "__main__":
    customers_orders = fetch_customers_orders(URI, AUTH)
    if customers_orders:
        print("Customer Orders:")
        print("-" * 80)
        for item in customers_orders:
            print(f"Name: {item['full_name']}")
            print(f"Email: {item['email']}")
            print(f"Order ID: {item['order_id']}")
            print(f"Order Date: {item['order_date']}")
            print(f"Products: {', '.join(item['product_list'])}")
            print(f"Total Paid: ${item['grand_total']:.2f}")
            print("-" * 80)
    else:
        print("No customer orders found.")



/var/folders/vk/cw2q21sj3l37fm6ls3k4g97c0000gq/T/ipykernel_55865/4259543511.py:15: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


Customer Orders:
--------------------------------------------------------------------------------
Name: Kelly Barnes
Email: lesliesanchez@example.org
Order ID: 113
Order Date: None
Products: Sense, No
Total Paid: $717.25
--------------------------------------------------------------------------------
Name: Ashley Brown
Email: emily40@example.com
Order ID: 121
Order Date: 2024-07-06T23:00:00.000000000+00:00
Products: Hair, Several
Total Paid: $2404.78
--------------------------------------------------------------------------------
Name: Krystal Carr
Email: lisa13@example.net
Order ID: 127
Order Date: 2024-01-01T00:00:00.000000000+00:00
Products: Company, Condition
Total Paid: $2458.77
--------------------------------------------------------------------------------
Name: Julie Cole
Email: cmiller@example.org
Order ID: 125
Order Date: 2024-10-03T23:00:00.000000000+00:00
Products: Reflect, Unit
Total Paid: $1865.97
---------------------------------------------------------------------------

Q2D)

In [48]:

def fetch_customer_basket_details(uri, auth):
    """Retrieve customers with items in their baskets, their birthdays, gift card balances, and products."""

    query = """
    MATCH (c:Customers)-[:has_a]->(b:Basket)-[:contains]->(p:Products)
    OPTIONAL MATCH (c)-[:saved]->(pm:Payment_Method)-[:is_a_gift]->(g:GiftCard)
    RETURN
      c.email AS email,
      c.first_name + " " + c.last_name AS full_name,
      c.birth_date AS birth_date,
      g.current_balance AS gift_card_balance,
      collect(p.Name) AS product_list
    ORDER BY c.email;
    """

    with driver.session() as session:
        result = session.run(query)
        data = []
        for record in result:
            # Append each result to the list as a structured dictionary
            data.append({
                "email": record["email"],
                "full_name": record["full_name"],
                "birth_date": record["birth_date"],
                "gift_card_balance": record["gift_card_balance"],
                "product_list": record["product_list"],
            })
    return data


if __name__ == "__main__":
    customers = fetch_customer_basket_details(URI, AUTH)
    # Print the results in a readable format
    if customers:
        print("Customers with items in their basket and gift card/voucher balances:")
        print("-" * 80)
        for customer in customers:
            print(f"Email: {customer['email']}")
            print(f"Name: {customer['full_name']}")
            print(f"Birthday: {customer['birth_date']}")
            gift_card_balance = customer['gift_card_balance'] or 0.0  # Default to 0.0 if None
            print(f"Gift Card Balance: ${gift_card_balance:.2f}")
            print(f"Products in Basket: {', '.join(customer['product_list'])}")
            print("-" * 80)
    else:
        print("No customers with items in their baskets were found.")



/var/folders/vk/cw2q21sj3l37fm6ls3k4g97c0000gq/T/ipykernel_55865/4231791255.py:16: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


Customers with items in their basket and gift card/voucher balances:
--------------------------------------------------------------------------------
Email: achang@example.org
Name: Jonathan Dixon
Birthday: 1963-06-03T00:00:00.000000000+00:00
Gift Card Balance: $0.00
Products in Basket: Hair, Several
--------------------------------------------------------------------------------
Email: brittany92@example.net
Name: Kyle Goodman
Birthday: 1994-05-16T00:00:00.000000000+00:00
Gift Card Balance: $0.00
Products in Basket: Reflect, Unit
--------------------------------------------------------------------------------
Email: brucesmith@example.com
Name: Angela Roberts
Birthday: 2001-01-12T00:00:00.000000000+00:00
Gift Card Balance: $0.00
Products in Basket: Sense, No
--------------------------------------------------------------------------------
Email: charlesturner@example.com
Name: Henry Gordon
Birthday: 1936-09-25T00:00:00.000000000+00:00
Gift Card Balance: $0.00
Products in Basket: Follow

Q2E)

In [70]:
def fetch_top_products_by_category(uri, auth):
    """
    Retrieve the top two items sold in each product category with their total sales.
    """
    query = """
    MATCH (o:Orders)-[r:order_item]->(p:Products)-[:is_a]->(cat:categories)
    WITH cat.Name_ AS category_name, p.Name AS product_name, SUM(r.quantity) AS total_sales
    ORDER BY category_name, total_sales DESC
    WITH category_name, COLLECT({product_name: product_name, total_sales: total_sales}) AS ranked_products
    RETURN
      category_name,
      ranked_products[0..2] AS top_two_products
    """

    with GraphDatabase.driver(uri, auth=auth) as driver:
        with driver.session() as session:
            results = session.run(query)
            data = []
            for record in results:
                # Format and structure the data for output
                data.append({
                    "category_name": record["category_name"],
                    "top_two_products": record["top_two_products"]
                })
    return data


if __name__ == "__main__":
    try:
        # Fetch the top products by category
        top_products = fetch_top_products_by_category(URI, AUTH)

        # Output the results
        print("Top Two Products by Category:")
        print("=" * 40)
        for category in top_products:
            print(f"Category: {category['category_name']}")
            for product in category['top_two_products']:
                print(f"  - Product: {product['product_name']}, Total Sales: {product['total_sales']}")
            print("=" * 40)
    except Exception as e:
        print(f"An error occurred: {e}")

driver.close()

Top Two Products by Category:
Category: Furniture
  - Product: Reflect, Total Sales: 24
  - Product: Pick, Total Sales: 14
Category: Kitchen Appliances
  - Product: Unit, Total Sales: 20
  - Product: Several, Total Sales: 18
Category: Office Supplies
  - Product: Condition, Total Sales: 20
  - Product: No, Total Sales: 13
Category: Outdoor Equipment
  - Product: Follow, Total Sales: 15
